In [1]:
import pandas as pd
import os
import numpy as np
from tqdm import tqdm #progress bar

/home/ns2dumon/anaconda3/lib/python3.7/site-packages/pandas/compat/_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)


In [2]:
def add_roll_mean(df):
    roll_lst=[]
    for col in df.columns[23:27]:
        #from df, the data
        temp_data = df[col]

        #create empty lists for recording new values
        lst=[]
        Roll_mean=[]

        #for each trial
        for i in range(len(temp_data)):
            x = temp_data[i]
            #the data was saved as a string so convert it to an array
            x = x.replace('[','')
            x = x.replace(']','')
            array = np.fromstring(x, dtype=float, sep=",")
            
            if col == 'episodes ':            
                #convert array to dataframe
                array_df = pd.DataFrame(array)

                #calculate rolling mean and add to one of the lists
                Roll_mean.append(array_df[array_df.columns[0]].rolling(100).mean())

                #copy the rolling mean data and convert to an array
                a = np.asarray(Roll_mean[i]).copy()
                #add the new array to the second list
                roll_lst.append(a)
            lst.append(array)

        #create new column with rolling mean data
        df[col] = lst
    
    df['roll_mean '] = roll_lst
    
    #spike_rows=[]
    #for i in range(len(df)):
    #    x = df['spikes '][i]
    #    x= x.replace(']', '')
    #    x= x.replace(' ', '')
    #    xSplit = x[:].split('[')
    #    spike_list=[]
    #    for j in range(3, len(xSplit)):  
    #        str_list = xSplit[j].split(',')
    #        arr_list=[]
    #        for i in str_list:
    #            try:
    #                arr_list.append(float(i))
    #            except ValueError:
    #                continue
    #        if len(arr_list) > 1:
    #            spike_list.append(np.asarray(arr_list))
    #    spike_rows.append(spike_list)
    #df['spikes ']=spike_rows
    
    #return dataframe
    return df

In [3]:
#Function for fetching data from the txt files containing the data from the NNI experiments
def txt_to_dataFrame(directory):
    i = 0
    #from the directory, grab each data file
    for filename in tqdm(os.listdir(directory)):
        #get path to data file
        filepath = os.path.join(directory, filename)

        #Read the data as a csv, separating the data names from the data 
        #Data format: 'dataname = data'
        data = pd.read_csv(filepath, sep='=', encoding='utf-8', header=None)
        
        #Store the data in list
        vals = data.iloc[:27,1].tolist()
        #vals.append(data.iloc[-1,1])

        #To initialise the dataframe
        if i == 0:

            #make sure the first file has all the data needed
            if len(vals) == 27:

                #Store the data names in a list
                head = data.iloc[:27,0].tolist()
                #head.append(data.iloc[-1,0])

                #Create a pandas data frame
                df = pd.DataFrame(vals)
                df = df.T

                #Set the column names
                df.columns = head

                i += 1

            #if the first file doesn't have all the data needed
            else:

                #print 'incomplete data' if the file is missing something
                #script will then move on to the next file
                print('Incomplete data in:',filepath)
        
        #for the rest of the data files, add the data to the dataframe
        else:
            try:
                df.loc[len(df.index)] = vals

            #if there is data missing, print 'incomplete data' and move to the next file
            except ValueError:
                print('incomplete data in:',filepath)
    
    df = add_roll_mean(df)
    
    return df

In [4]:
## Path to folder containing individual folders for each NNI experiment
folder = './gridsearchData'

In [5]:
## For each experiment, extract data and save it as a pandas dataframe
df = txt_to_dataFrame(folder)

df['env '][:] = 'MiniGrid Empty 8x8'
df['rule '][:] = 'TD0'
df['rep '][:] = 'Grid Cells'

#Calculate the mean number of runs to reach the goal rolling average reward
goal = 0.95
goal_reached = []
for i in range(len(df)):
    a=[i for i,v in enumerate(df['roll_mean '][i]) if np.abs(v) > goal]
    if len(a) < 1:
        goal_reached.append(np.nan)
    else:
        goal_reached.append(a[0])
        
goal_reached = np.asarray(goal_reached)
df['goal_in_n '] = goal_reached
    
df.to_pickle('./gridsearch_dataframe')

FileNotFoundError: [Errno 2] No such file or directory: './gridsearchData'

## Split Dataframe into Smaller Chunks

In [ ]:
import pandas as pd

In [ ]:
data = pd.read_pickle('.\\gridsearch_dataframe')

In [ ]:
len(data)/4

In [ ]:
data1 = data.iloc[:245]
data2 = data.iloc[245:245*2]
data3 = data.iloc[245*2:245*3]
data4 = data.iloc[245*3:]

In [ ]:
data1.to_pickle('./gridsearch_dataframe_first')
data2.to_pickle('./gridsearch_dataframe_second')
data3.to_pickle('./gridsearch_dataframe_third')
data4.to_pickle('./gridsearch_dataframe_fourth')

## Join Data Together

In [2]:
import pandas as pd
import pickle5 as pickle

In [3]:
with open('gridsearch_dataframe_first', 'rb') as pickle_file:
    content = pickle.load(pickle_file)

AttributeError: Can't get attribute '_unpickle_block' on <module 'pandas._libs.internals' from '/home/ns2dumon/anaconda3/lib/python3.7/site-packages/pandas/_libs/internals.cpython-37m-x86_64-linux-gnu.so'>

In [8]:
data1 = pd.read_pickle('gridsearch_dataframe_first')
data2 = pd.read_pickle('gridsearch_dataframe_second')
data3 = pd.read_pickle('gridsearch_dataframe_third')
data4 = pd.read_pickle('gridsearch_dataframe_fourth')

ValueError: unsupported pickle protocol: 5

In [ ]:
data_tog = pd.concat([data1, data2, data3, data4], ignore_index=True)

In [ ]:
data_tog.to_pickle('./gridsearch_dataframe')